In [ ]:
#question 1

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
import pickle
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from scipy.stats import randint, uniform

In [51]:
male_players = pd.read_csv('/Users/jamesck/Desktop/Intro to AI/data/male_players (legacy).csv', na_values='-',low_memory=False)

In [52]:
male_players.head()

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png


In [53]:
def drop_low_correlation(overall_dataframe, target_column, threshold=0.5):
    numerical_data = overall_dataframe.select_dtypes(include=['number'])
    columns_to_drop = []

    for column in numerical_data.columns:
        if column != target_column:
            corr = numerical_data[column].corr(numerical_data[target_column])
            if abs(corr) < threshold or pd.isna(corr):
                columns_to_drop.append(column)

    overall_dataframe = overall_dataframe.drop(columns=columns_to_drop)
    return overall_dataframe

result = drop_low_correlation(male_players, 'overall')
result.head()

/opt/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


,player_url,fifa_update_date,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,dob,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,/player/158023/lionel-messi/150002,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,100500000.0,550000.0,1987-06-24,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,/player/20801/c-ronaldo-dos-santos-aveiro/150002,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,79000000.0,375000.0,1985-02-05,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,/player/9014/arjen-robben/150002,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,54500000.0,275000.0,1984-01-23,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,/player/41236/zlatan-ibrahimovic/150002,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,52500000.0,275000.0,1981-10-03,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,/player/167495/manuel-neuer/150002,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,63500000.0,300000.0,1986-03-27,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png


In [67]:
# drop columns With More than 30 Percent Null Values

def drop_big_null_columns(data, threshold=0.3):
    thresh_count = int((1 - threshold) * len(data))
    data_cleaned = data.dropna(axis=1, thresh=thresh_count)
    return data_cleaned

result = drop_big_null_columns(result, threshold=0.3)

In [68]:
result.head()

,player_url,fifa_update_date,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,dob,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,/player/158023/lionel-messi/150002,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,100500000.0,550000.0,1987-06-24,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,/player/20801/c-ronaldo-dos-santos-aveiro/150002,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,79000000.0,375000.0,1985-02-05,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,/player/9014/arjen-robben/150002,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,54500000.0,275000.0,1984-01-23,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,/player/41236/zlatan-ibrahimovic/150002,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,52500000.0,275000.0,1981-10-03,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,/player/167495/manuel-neuer/150002,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,63500000.0,300000.0,1986-03-27,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png


In [69]:
df = pd.DataFrame(male_players)

In [70]:
#drop all categorical columns
df = df.select_dtypes(exclude=['object'])

In [71]:
# Function to impute the median value where there are NaN values
def median_imputing(dataframes):
    numeric_cols = dataframes.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        column_median = dataframes[col].median(skipna=True)
        dataframes[col] = dataframes[col].fillna(column_median)
    return dataframes

result = median_imputing(result)

In [72]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 52 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   player_url           161583 non-null  object 
 1   fifa_update_date     161583 non-null  object 
 2   short_name           161583 non-null  object 
 3   long_name            161583 non-null  object 
 4   player_positions     161583 non-null  object 
 5   overall              161583 non-null  int64  
 6   potential            161583 non-null  int64  
 7   value_eur            161583 non-null  float64
 8   wage_eur             161583 non-null  float64
 9   dob                  161583 non-null  object 
 10  league_name          159810 non-null  object 
 11  club_name            159810 non-null  object 
 12  club_position        159810 non-null  object 
 13  club_joined_date     150272 non-null  object 
 14  nationality_name     161583 non-null  object 
 15  preferred_foot   

In [73]:
#Feature Engineering

In [ ]:
rom sklearn.preprocessing import StandardScaler

def scale_independent_variables(dataframe, target_column):
    df_scaled = dataframe.copy()
    numerical_columns = [col for col in df_scaled.columns if df_scaled[col].dtype.kind in ['i', 'f']]
    numerical_columns = [col for col in numerical_columns if col!= target_column]
    scaler = StandardScaler()
    df_scaled[numerical_columns] = scaler.fit_transform(df_scaled[numerical_columns])

    return df_scaled, scaler

scaled_df, scaler = scale_independent_variables(result, 'overall')
result = scaled_df

In [ ]:
# separating and labelling
X = result.drop("overall",axis=1) # Features
y = result["overall"] # Labels

In [ ]:
# Splitting into training and testing 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)
X_train.head()

In [ ]:
# Combine the preprocessed features and target for correlation analysis
train_data = pd.concat([X_train, y_train.reset_index(drop=True)], axis=1)

# Calculate correlation matrix
corr_matrix = train_data.corr()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": xgb.XGBRegressor(n_estimators=100, random_state=42),
    "GradientBoost": GradientBoostingRegressor(n_estimators=100, random_state=42)
}

In [ ]:
for model_name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
    print(f"{model_name} Cross-Validation R2 Scores: {scores}")
    print(f"{model_name} Mean Cross-Validation R2 Score: {np.mean(scores)}")
    model.fit(X_train, y_train)
    test_score = model.score(X_test, y_test)
    print(f"{model_name} Test R2 Score: {test_score}\n")

In [ ]:
# Linear Regression

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
Linear_model = LinearRegression()
Linear_model.fit(X_train, y_train)
y_pred_test = Linear_model.predict(X_test)
Linear_test_r2 = r2_score(y_test, y_pred_test)
Linear_test_mae = mean_absolute_error(y_test, y_pred_test)


In [ ]:
# Decision Tree Regression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
Decision_Tree_model = DecisionTreeRegressor(random_state=42)
Decision_Tree_model.fit(X_train, y_train)
Decision_Tree_y_pred_test = Decision_Tree_model.predict(X_test)
Decision_Tree_test_r2 = r2_score(y_test, Decision_Tree_y_pred_test)
Decision_Tree_test_mae = mean_absolute_error(y_test, Decision_Tree_y_pred_test)


In [ ]:
# Using evaluation metrics to assess our basic models

#Decision Tree Regression
mae = mean_absolute_error(y_test, y_pred_test)
print("Decision Tree Regression without hyperparameters test:")
print(f"Test R2 score: {Decision_Tree_test_r2}")
print(f"Test MAE: {Decision_Tree_test_mae}")

In [ ]:
# Evaluation On Linear Regression

# Linear Regression
print("Linear Regression without hyperparameters:")
print(f"Test R2 score: {Linear_test_r2}")
print(f"Test MAE: {Linear_test_mae}")

In [ ]:
# Using evaluation metrics to assess our ensemble models
for model_name, model in models.items():
    model.fit(X_train, y_train)

    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    test_r2 = model.score(X_test, y_test)
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)

    print(f"{model_name} Train MAE: {train_mae}")
    print(f"{model_name} Test MAE: {test_mae}")
    print(f"{model_name} Test R2 Score: {test_r2}\n")

In [ ]:
# Hypertuning best ensemble model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xgb_model = models["XGBoost"]

param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
}

random_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_dist,
    n_iter=12,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train, y_train)

print(f"Best parameters for XGBoost: {random_search.best_params_}")
print(f"Best cross-validation R2 score for XGBoost: {random_search.best_score_}")

best_model = random_search.best_estimator_

best_model.fit(X_train, y_train)
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)

In [ ]:
# Evaluating hypertuned model
print(f"XGBoost Train R2 Score: {train_r2}")
print(f"XGBoost Test R2 Score: {test_r2}")
print(f"XGBoost Train MAE: {train_mae}")
print(f"XGBoost Test MAE: {test_mae}")

In [ ]:
#LOADING TEST DATA SET
players_22 = pd.read_csv('/Users/jamesck/Desktop/Intro to AI/data/players_22.csv')
players_22.head()

In [ ]:
# Dropping Columns with Low Correlation in The Testing Dataset
players_22 = drop_low_correlation(players_22, 'overall')
# Dropping The Columns With Null Values greater than 30 percent of the total column
players_22 = drop_big_null_columns(players_22, threshold=0.3)

In [ ]:
# Dropping Categorical Columns From This DataSet
df = pd.DataFrame(players_22)
df = df.select_dtypes(exclude=['object'])

In [ ]:
#imputing to fill the null results with the median of the columns
players_22 = median_imputing(players_22)

In [ ]:
# Scaling the independent variables 
scaled_df, scaler = scale_independent_variables(players_22, 'overall')
players_22 = scaled_df

In [ ]:
players_22

In [ ]:
#only columns in the training data set are available in the testing data set
common_columns = list(set(result.columns) & set(players_22.columns))
filtered_players_22 = players_22[common_columns]

In [ ]:
#Testing The Finished Model

In [ ]:
# Separating features from labels
X_test, y_test = filtered_players_22, filtered_players_22['overall']
X_test = X_test.drop('overall', axis=1)

In [ ]:
X_test_encoded = pd.get_dummies(X_test, columns=['rb', 'rm', 'cm','long_name','nationality_name', 'cam', 'body_type', 'rw', 'short_name', 'lm', 'club_position','lw', 'lf', 'cdm', 'rwb', 'preferred_foot', 'ldm', 'rcb', 'rf', 'cb', 'rcm', 'gk', 'ram', 'lwb', 'dob', 'player_positions', 'league_name', 'lb', 'rs', 'rdm', 'player_face_url', 'ls', 'lam', 'cf', 'lcm', 'work_rate', 'club_name', 'player_url', 'real_face', 'lcb','st'])
y_pred = best_model.predict(X_test_encoded)

In [ ]:
# Make predictions
y_pred = best_model.predict(X_test)

In [ ]:
# Evaluate model's performance
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared Score: {r2}")

In [ ]:
# Using Feature Importance To Identify Features For Webapp

In [ ]:
feature_importances = best_model.feature_importances_

feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 8))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.gca().invert_yaxis()
plt.show()

top_features = feature_importance_df.head(10)['Feature']
print("Top Features:")
print(top_features)

In [ ]:
important_features = ['value_eur', 'movement_reactions', 'wage_eur', 'physic', 'potential']

# Subset the original dataset to include both important features and target variable
X_subset = players_dataset[important_features + ['overall']]
scalernew = StandardScaler()

X = X_subset.drop(columns=['overall'])
y = X_subset['overall']

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Fit and transform the scaler on the training data
X_train_scaled = scalernew.fit_transform(X_train)
# Transform the test data using the fitted scaler
X_test_scaled = scalernew.transform(X_test)

In [ ]:
# Train the model
best_model.fit(X_train_scaled, y_train)

In [ ]:
# Make predictions on training set
y_pred_train = best_model.predict(X_train_scaled)


In [ ]:
# Make predictions on test set
y_pred_test = best_model.predict(X_test_scaled)

In [ ]:
# Evaluate the model
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)

print(f"Train R2 Score: {train_r2}")
print(f"Test R2 Score: {test_r2}")
print(f"Train MAE: {train_mae}")
print(f"Test MAE: {test_mae}")

In [ ]:
# Downloading Scalar Used On Training and Test Data To Be Applied Onto User's Input
with open('scalernew.pkl', 'wb') as file:
    pickle.dump(scalernew, file)

In [122]:
with open('best_model', 'wb') as file:
    pickle.dump(best_model, file)

In [125]:
import streamlit as st
import pandas as pd
import numpy as np
import joblib

# Load the model and preprocessor
best_model = joblib.load('best_model')
preprocessor = joblib.load('preprocessor')
relevant_features = joblib.load('relevant_features')

st.title('FIFA Rating Predictor')

FileNotFoundError: [Errno 2] No such file or directory: 'preprocessor'

In [124]:
uploaded_file = st.file_uploader('Upload a CSV file', type=['csv'])

if uploaded_file is not None:
    data = pd.read_csv(uploaded_file)

    # Apply the same preprocessing steps
    data_preprocessed = preprocessor.transform(data)
    data_relevant = pd.DataFrame(data_preprocessed, columns=preprocessor.get_feature_names_out())[relevant_features]

    # Predict
    predictions = best_model.predict(data_relevant)

    # Display predictions
    st.write("Predicted Ratings:")
    st.write(predictions)

    # Display confidence scores (standard deviation of the predictions)
    confidence_scores = np.std(predictions)
    st.write("Confidence Score:")
    st.write(confidence_scores)

2024-06-24 12:54:02.151 
  command:

    streamlit run /opt/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
